In [1]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login
import nltk
import evaluate
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

#interpreter_login()

In [2]:
import torch

torch.cuda.is_available()

torch.cuda.current_device()

torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4070 Laptop GPU'

In [3]:
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
movie_dataset = load_dataset("Pablinho/movies-dataset")

In [5]:
movie_dataset


DatasetDict({
    train: Dataset({
        features: ['Release_Date', 'Title', 'Overview', 'Popularity', 'Vote_Count', 'Vote_Average', 'Original_Language', 'Genre', 'Poster_Url'],
        num_rows: 9837
    })
})

In [19]:
movie_dataset = movie_dataset.remove_columns(["Popularity","Vote_Count","Vote_Average","Original_Language","Poster_Url"])

ValueError: Column name ['Popularity', 'Poster_Url', 'Vote_Average', 'Vote_Count', 'Original_Language'] not in the dataset. Current columns in the dataset: ['Release_Date', 'Title', 'Overview', 'Genre', 'full_question']

In [6]:
# We prefix our tasks with "Please answer the question" since this is how inferences are fed to the model later.
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["full_question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [8]:
# use the map function to adjust the full_question value of the dataset

def concat_features(movie):
    """
    adjust label lengths if they dont match.
    """
    question = "Please recommend a movie from " + str(movie['Release_Date']) + " of the genre " + str(movie['Genre']) + " with a plot like: " + str(movie['Overview'])
    movie["full_question"] = question
    return movie

movie_dataset = movie_dataset.map(concat_features)

Map:   0%|          | 0/9837 [00:00<?, ? examples/s]

In [9]:
movie_dataset = movie_dataset["train"].train_test_split(test_size=0.3)
movie_dataset

DatasetDict({
    train: Dataset({
        features: ['Release_Date', 'Title', 'Overview', 'Popularity', 'Vote_Count', 'Vote_Average', 'Original_Language', 'Genre', 'Poster_Url', 'full_question'],
        num_rows: 6885
    })
    test: Dataset({
        features: ['Release_Date', 'Title', 'Overview', 'Popularity', 'Vote_Count', 'Vote_Average', 'Original_Language', 'Genre', 'Poster_Url', 'full_question'],
        num_rows: 2952
    })
})

In [29]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["full_question"]]
   model_inputs = tokenizer(inputs, max_length=512, truncation=True,return_overflowing_tokens=True,padding=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=str(examples["Title"]), 
                      max_length=512,         
                      truncation=True,
                    padding=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [30]:
# Map the preprocessing function across our dataset
tokenized_dataset = movie_dataset.map(preprocess_function)


Map:   0%|          | 0/6885 [00:00<?, ? examples/s]

Map:   0%|          | 0/2952 [00:00<?, ? examples/s]

In [22]:
# calculate rouge score

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

In [23]:
# training

# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   eval_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [31]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

C:\Users\gruau\AppData\Local\Temp\ipykernel_58348\2420821063.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [34]:
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).